In [120]:
import ollama

from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.chunking import HybridChunker

from pydantic.v1 import BaseModel

from transformers import AutoTokenizer

import json

In [121]:
# document local path or URL
doc = "./test_doc/test_textMore.pdf" # simple text with a image and table
doc1 = "./test_doc/吉美利101利率變動型美元終身壽險(定期給付型)DM.pdf" # complex text with simple aesthetic table
doc2 = "./test_doc/美利雙寶利率變動型美元終身保險 (定期給付型)DM.pdf" # complex text with complex aesthetic table
doc3 = "./test_doc/國泰金控出勤管理須知(修正後).pdf" # simple text with a simple table
doc4 = "./test_doc/國泰金控員工國內出差要點.pdf" # simple text with a simple table
doc5 = "./test_doc/國泰金控員工國外出差要點.pdf" # simple text with complex table
doc6 = "./test_doc/國泰金融控股股份有限公司資訊安全管理辦法.pdf" # simple text

# ollama embedding model
embed_model = "bge-m3:latest"

# ollama llm
llm = "deepseek-r1:7b"

def get_embeddings(texts, model=embed_model):
    embed_response = ollama.embeddings(model=model, prompt=texts)
    embedded_vector = embed_response["embedding"]
    
    return embedded_vector

# PyPdfium without EasyOCR
# --------------------
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False
#pipeline_options.table_structure_options.do_cell_matching = True

#pipeline_options.table_structure_options.mode = 'accurate'

pipeline_options.images_scale = 2
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

pypdfium_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
        )
    }
)

# print json
def show_json(data):
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4, ensure_ascii=False))
    elif isinstance(data, dict) or isinstance(data, list):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict(), indent=4, ensure_ascii=False))

# Docling reader, docling format parser output

In [122]:
doc_source = [doc, doc1, doc2, doc3, doc4, doc5, doc6]

conv_results = pypdfium_converter.convert_all(
    doc_source,
    raises_on_error=True,  # to let conversion run through all and examine results at the end
)

# Do hybrid chunking to merge similar chunk
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hybrid_chunker = HybridChunker(
    tokenizer=tokenizer,
    merge_peers=True,  # optional, defaults to True
)

all_chunks = []
for conv_res in conv_results:
    docling_docs = conv_res.document
    chunk_iter = hybrid_chunker.chunk(dl_doc=docling_docs)
    chunks = list(chunk_iter)
    all_chunks += chunks

In [123]:
for i, chunk in enumerate(all_chunks[:50]):
    print(f"=== {i} ===")
    txt_tokens = len(tokenizer.tokenize(chunk.text))
    print(f"chunk.text ({txt_tokens} tokens):\n{repr(chunk.text)}")

    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")

    print()

=== 0 ===
chunk.text (285 tokens):
'（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正'
chunker.serialize(chunk) (311 tokens):
'國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）\n（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 

In [124]:
node_text, node_metadatas = [], []
for chunk in all_chunks:
    node_text.append(hybrid_chunker.serialize(chunk=chunk))
    node_metadatas.append(chunk.meta.export_json_dict())

In [125]:
for i, (text, metadata) in enumerate(zip(node_text[:50], node_metadatas[:50])):
    print(f"=== {i} ===")
    print(f"### Text Section ###\n")
    print(f"{text}\n")
    print(f"### Metadata Section ###")
    print(f"{metadata}\n")

=== 0 ===
### Text Section ###

國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）
（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）
（本保險提供身故保險金分期定期給付）
（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）
（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）
（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）
（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：
service@cathaylife.com.tw）
110.08.12國2國壽字第1100080120號0號函備查
111 . 12 . 01國1國壽字第1110120022號2號函備查
112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正
113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正
113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正

### Metadata Section ###
{'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/1', 'parent': {'$ref': '#/groups/0'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 61.619998931884766, 't': 745.7193603515625, 'r': 391.8338623046875, 'b': 736.2374267578125, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 34]}]}, {'self_ref': '#/texts/2', 'parent': {'$ref': '#/groups/0'

In [126]:
embedded_text = []
for i in node_text:
    embedded_text.append(get_embeddings(i))

In [127]:
for i, vector in enumerate(embedded_text[:50]):
    print(f"=== {i} ===")
    print(vector)

=== 0 ===
[0.2827550768852234, 0.1642068326473236, -0.09033191204071045, -0.9640346765518188, -1.3323609828948975, -1.3135981559753418, 1.4667359590530396, -0.27096420526504517, 0.7208428978919983, -0.22803029417991638, 0.8666432499885559, 0.3868962526321411, -0.9509731531143188, 0.5981506705284119, -0.00889337994158268, -0.4897511601448059, -0.28481119871139526, -0.7278636693954468, -1.9225914478302002, -0.5729472637176514, 0.08194125443696976, -0.029961181804537773, -0.8877862095832825, 0.17574286460876465, -0.3429093658924103, -0.2959047257900238, -0.6203485131263733, -0.6503872275352478, -0.2060478925704956, 0.28371286392211914, 0.09690633416175842, 0.30712243914604187, -0.28054043650627136, -1.0088932514190674, -0.8582723140716553, 0.8984909057617188, -1.007388710975647, 0.04417720437049866, -1.2611563205718994, 0.903727114200592, 1.4088513851165771, 0.6094723343849182, 1.6675959825515747, 0.34138911962509155, -0.1193062961101532, -0.588374137878418, 0.4352114498615265, -0.0300594

# Qdrant


In [128]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct

class qdrant_DBConnector:
    def __init__(self, collection_name):#, embedding_fn):
        self.qdrant_client = QdrantClient("http://localhost:6333")

        self.collection_name = collection_name

        # create collection
        self.collection = self.qdrant_client.recreate_collection(
            collection_name = collection_name,
            vectors_config = models.VectorParams(
                distance = models.Distance.COSINE,
                size=len(get_embeddings("你好"))),
            optimizers_config = models.OptimizersConfigDiff(memmap_threshold=20000),
            hnsw_config = models.HnswConfigDiff(on_disk=True, m=16, ef_construct=100)
        )
        '''
        # embed models
        self.embedding_fn = embedding_fn
        qdrant_client.set_model(self.embedding_fn)
        '''

    ''' WARNING: NO CHECK ON EQUAL LENGTH YET'''
    def upsert_vector(self, vectors, data):
        # insert 'points' to qdrant by vector, 
        # payload with original text and metadata
        for i, vector in enumerate(vectors):
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(id=i,
                                    vector=vectors[i],
                                    payload={"text": data.text[i],
                                             "metadata": data.metadata[i]})]
                                    
            )

        print("upsert finish")

    def search(self, vector, top_k):
        # vector search qdrant DB
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        return result

In [129]:
class DataObject:
    def __init__(self, text, metadata):
        self.text = text
        self.metadata = metadata

data = DataObject(node_text, node_metadatas)
data.text[0]

'國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）\n（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正'

In [131]:
# create a qdrant vectorDB object
vector_db = qdrant_DBConnector("qdrant_test")
# add data to db
vector_db.upsert_vector(embedded_text, data)

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_49292/1177792719.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.collection = self.qdrant_client.recreate_collection(


upsert finish


In [132]:
#query = "國內出差費用標準"
#query = "國外出差，附表五 "
query = "系統開發之安全管理，應包含哪些項目？"

embedded_query = get_embeddings(query)
results = vector_db.search(embedded_query, 5)
print(f"尋找 「{query}」:")
#print(results[0])

for i, point in enumerate(results):
    print(f"\n🔹 Score: {point.score:.2f} - point ID: {point.id}:")
    print(point.payload['text'])
    print()


尋找 「系統開發之安全管理，應包含哪些項目？」:

🔹 Score: 0.51 - point ID: 109:
用詞定義
第三條 本辦法相法相關用詞定義如下：
第 1 頁1 頁/共 8 頁8 頁
資訊安全：目的在確保資訊的機密性、完整性、可用性、 私密性及合法性 ， 使資訊能安 全地 、正確地、適切地 及 可靠地被運用在達成本公司經 營目 標之規劃、執行、 管 理及相關作為上。


🔹 Score: 0.51 - point ID: 107:
主旨
第-條 為強為強化國化國泰金融控股股份有限公司(以下簡稱本公司)之資訊安全管 理，建立安全及可信賴之資訊作業環境，確保資保資訊財訊財產之機密性、完 整性及可用性，並提升同仁對資訊安全之認知， 以保以保障員障員工、客戶 、 合作夥伴與本公司之權益，依據｢國泰金融控股股份有限公司資訊安 全政策｣訂｣訂定資訊安全管理辦法(以下簡稱本辦法)。


🔹 Score: 0.49 - point ID: 106:
國泰金融控股股份有限公司資訊安全管全管理辦法
110年3月3月22日2日訂定 權責單位：資訊安全部


🔹 Score: 0.46 - point ID: 108:
適用對象及範及範圍
第二條 本辦法適法適用對象為本公司所有員工(含(含各級主管、實、實習生及工讀生，以 下同)及)及其他得接觸本公司資訊及訊及資訊財訊財產之合之合作夥伴、委託廠商(含(含 顧問) (以下合稱「受「受委託者」)等。
本辦法適用範用範圍如下，以下合稱資訊財訊財產(Information Assets)：
-、 本公司所有或有或持有之資之資料與文件。
二、 本公司使用之主機、伺服器、個人電腦 (含(含可攜式電腦，以 下同)、行動裝置、終端設備、通訊線路 、儲存設備(含(含可 讀寫CD裝置、外接硬碟 、 USB儲B儲存媒體、記憶卡等，以 下同)，以及與前述相關或是與本公司資司資訊系統相連之硬 體資訊財訊財產。
三、 本公司使用於存於存放資訊財訊財產之實體環境。
四、 本公司使用之資之資訊系統，以及與前述資述資訊系統相關或相 連之軟體資訊財訊財產。
五、 其他本公司所有或有或有權使權使用之資之資訊財訊財產。


🔹 Score: 0.46 - point ID: 73:
第十條十條（控管機制）
各單位主管應督導所屬確實遵守本須知，並掌握員工出、缺勤異常動

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_49292/1177792719.py:44: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.qdrant_client.search(


# FAISS retrieval

In [133]:
'''
There seems to be some unsolved conflict between Docling and Faiss.
just <pip install fails-cpu> after docling parsing and chunking phase is over.
'''

import numpy as np
import faiss

def add_to_faiss_index(embeddings):
    vector = np.array(embeddings)
    index = faiss.IndexFlatL2(vector.shape[1])
    index.add(vector)
    return index

def vector_search(index, query_embedding, text_array, top_k=3):
    distances, indices = index.search(np.array([query_embedding]), top_k)
    return [(text_array[i], float(dist)) for dist, i in zip(distances[0], indices[0])]

In [134]:
faiss_index = add_to_faiss_index(embedded_text)

In [135]:
#query = "國內出差費用標準"
#query = "國外出差，附表五 "
query = "系統開發之安全管理，應包含哪些項目？"

embedded_query = get_embeddings(query)
search_results = vector_search(faiss_index, embedded_query, node_text, top_k=5)

for i, result in enumerate(search_results):
    print(f"\n🔹 Distance: {result[1]:.2f} - point: {i}:")
    print(result[0])
    print()



🔹 Distance: 667.48 - point: 0:
用詞定義
第三條 本辦法相法相關用詞定義如下：
第 1 頁1 頁/共 8 頁8 頁
資訊安全：目的在確保資訊的機密性、完整性、可用性、 私密性及合法性 ， 使資訊能安 全地 、正確地、適切地 及 可靠地被運用在達成本公司經 營目 標之規劃、執行、 管 理及相關作為上。


🔹 Distance: 680.81 - point: 1:
主旨
第-條 為強為強化國化國泰金融控股股份有限公司(以下簡稱本公司)之資訊安全管 理，建立安全及可信賴之資訊作業環境，確保資保資訊財訊財產之機密性、完 整性及可用性，並提升同仁對資訊安全之認知， 以保以保障員障員工、客戶 、 合作夥伴與本公司之權益，依據｢國泰金融控股股份有限公司資訊安 全政策｣訂｣訂定資訊安全管理辦法(以下簡稱本辦法)。


🔹 Distance: 717.04 - point: 2:
國泰金融控股股份有限公司資訊安全管全管理辦法
110年3月3月22日2日訂定 權責單位：資訊安全部


🔹 Distance: 734.21 - point: 3:
適用對象及範及範圍
第二條 本辦法適法適用對象為本公司所有員工(含(含各級主管、實、實習生及工讀生，以 下同)及)及其他得接觸本公司資訊及訊及資訊財訊財產之合之合作夥伴、委託廠商(含(含 顧問) (以下合稱「受「受委託者」)等。
本辦法適用範用範圍如下，以下合稱資訊財訊財產(Information Assets)：
-、 本公司所有或有或持有之資之資料與文件。
二、 本公司使用之主機、伺服器、個人電腦 (含(含可攜式電腦，以 下同)、行動裝置、終端設備、通訊線路 、儲存設備(含(含可 讀寫CD裝置、外接硬碟 、 USB儲B儲存媒體、記憶卡等，以 下同)，以及與前述相關或是與本公司資司資訊系統相連之硬 體資訊財訊財產。
三、 本公司使用於存於存放資訊財訊財產之實體環境。
四、 本公司使用之資之資訊系統，以及與前述資述資訊系統相關或相 連之軟體資訊財訊財產。
五、 其他本公司所有或有或有權使權使用之資之資訊財訊財產。


🔹 Distance: 742.16 - point: 4:
第十條十條（控管機制）
各單位主管應督導所屬確實遵守本須知，並掌握員工出、缺勤異常動態。



# MongoDB

In [136]:
import os
import pymongo

def get_mongo_client(MONGODB_URI):
    """Establish and validate connection to the MongoDB."""

    client = pymongo.MongoClient(
        MONGODB_URI, appname="cathayTest_RAG.python"
    )

    # Validate the connection
    ping_result = client.admin.command("ping")
    if ping_result.get("ok") == 1.0:
        # Connection successful
        print("Connection to MongoDB successful")
        return client
    else:
        print("Connection to MongoDB failed")
    return None

In [137]:
MONGODB_URI = os.getenv("MONGODB_URI")
if not MONGODB_URI:
    print("MONGO_URI not set in environment variables")
    
mongo_client = get_mongo_client(MONGODB_URI)

DB_NAME = "RAG_database"
COLLECTION_NAME = "test_docs"

# Create or get the database
db = mongo_client[DB_NAME]

# Create or get the collections
collection = db[COLLECTION_NAME]

Connection to MongoDB successful


In [138]:
collection.delete_many({})

DeleteResult({'n': 113, 'electionId': ObjectId('7fffffff0000000000000018'), 'opTime': {'ts': Timestamp(1741769859, 17), 't': 24}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1741769859, 17), 'signature': {'hash': b'\x1f\x06\xa2Q\xef\xe2{\xf2\x0e\x87K\xad\x19\xc9\xb2\x1c\xf1\xd9\x88J', 'keyId': 7438652003963633666}}, 'operationTime': Timestamp(1741769859, 17)}, acknowledged=True)

In [139]:
'''OverflowError: MongoDB can only handle up to 8-byte ints'''
# thus turn binary_hash in metadata to string
for i, dict in enumerate(node_metadatas):
    if "binary_hash" in node_metadatas[i]['origin']:
        node_metadatas[i]['origin']["binary_hash"] = str(node_metadatas[i]['origin']["binary_hash"])  # turn to str

In [140]:
''' WARNING: NO CHECK ON EQUAL LENGTH YET'''
data_list = [
    {
        "embedding": embedded_text[i],
        "text": node_text[i],
        "metadata": node_metadatas[i]
    }
    for i in range(len(embedded_text))
]

''' mongoDB requires python list so no need
# transform to json
json_data_list = [json.dumps(data, ensure_ascii=False, indent=4) for data in data_list]
#show_json(json_data_list[0])
'''

# insert all data
collection.insert_many(data_list)

'''
# insert one by one for debug purpose
from pymongo.errors import OperationFailure
for i, data in enumerate(data_list):
    try:
        collection.insert_one(data)
        print(f"✅ 第 {i+1} 筆數據成功插入")
    except (OverflowError, OperationFailure) as e:
        print(f"❌ 第 {i+1} 筆數據插入失敗：{e}")
        print("👉 有問題的數據：", data)
'''

'\n# insert one by one for debug purpose\nfrom pymongo.errors import OperationFailure\nfor i, data in enumerate(data_list):\n    try:\n        collection.insert_one(data)\n        print(f"✅ 第 {i+1} 筆數據成功插入")\n    except (OverflowError, OperationFailure) as e:\n        print(f"❌ 第 {i+1} 筆數據插入失敗：{e}")\n        print("👉 有問題的數據：", data)\n'

In [141]:
# Do indexing

import time

from pymongo.operations import SearchIndexModel

def setup_vector_search_index(collection, index_definition, index_name="vector_index"):
    # setup a vector search index for a MongoDB collection and wait for 30 seconds.
    
    new_vector_search_index_model = SearchIndexModel(
        definition=index_definition, name=index_name, type="vectorSearch"
    )

    # Create the new index
    try:
        result = collection.create_search_index(model=new_vector_search_index_model)
        print(f"Creating index '{index_name}'...")

        # Sleep for 30 seconds
        print(f"Waiting for 30 seconds to allow index '{index_name}' to be created...")
        time.sleep(30)

        print(f"30-second wait completed for index '{index_name}'.")
        return result

    except Exception as e:
        print(f"Error creating new vector search index '{index_name}': {e!s}")
        return None
    
def create_vector_index_definition(dimensions, feild_name):
    return {
        "fields": [
            {
                "type": "vector",
                "path": feild_name,
                "numDimensions": dimensions,
                "similarity": "cosine",
            }
        ]
    }
     

In [142]:
collection.drop_search_index("vector_index")
dim = len(get_embeddings("你好"))
vector_index_definition = create_vector_index_definition(dim, "embedding")
setup_vector_search_index(collection, vector_index_definition, "vector_index")

Error creating new vector search index 'vector_index': An index named "vector_index" is already defined for collection test_docs. Index names must be unique for a source collection and all its views., full error: {'ok': 0.0, 'errmsg': 'An index named "vector_index" is already defined for collection test_docs. Index names must be unique for a source collection and all its views.', 'code': 68, 'codeName': 'IndexAlreadyExists', '$clusterTime': {'clusterTime': Timestamp(1741769865, 4), 'signature': {'hash': b';O\x99ghMa\x8a\x83\x0fx\x94\xc6ek\xdc#J\xa8r', 'keyId': 7438652003963633666}}, 'operationTime': Timestamp(1741769865, 4)}


In [143]:
def vector_search(user_query, top_k=150):
    # perform vector search in the MongoDB collection based on user query.

    # Generate embedding for the user query
    query_embedding = get_embeddings(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": top_k,  # Number of candidate matches to consider
            "limit": 5,  # Return top 5 matches
        }
    }

    # you CAN NOT use 1 & 0 at the same time choose ONLY one
    project_stage = {
        "$project": {
            #"_id": 0,  # Exclude _id field
            "embedding": 0,  # Exclude embedding field
            #"text": 1,  # Include text field
            #"metadata": 0,  # Include metadata field
            "score": {"$meta": "vectorSearchScore"},  # Include the search score
        }
    }

    pipeline = [vector_search_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [144]:
#query = "國內出差費用標準"
#query = "國外出差，附表五 "
query = "系統開發之安全管理，應包含哪些項目？"

result = vector_search(query)
print(f"尋找 「{query}」:")

for i, data in enumerate(result):
    print(f"\n🔹 Score: {data['score']:.2f}:")
    print(f"Text: {data['text']}")
    #print(f"Text: {data['metadata']}")
    print()

尋找 「系統開發之安全管理，應包含哪些項目？」:

🔹 Score: 0.76:
Text: 用詞定義
第三條 本辦法相法相關用詞定義如下：
第 1 頁1 頁/共 8 頁8 頁
資訊安全：目的在確保資訊的機密性、完整性、可用性、 私密性及合法性 ， 使資訊能安 全地 、正確地、適切地 及 可靠地被運用在達成本公司經 營目 標之規劃、執行、 管 理及相關作為上。


🔹 Score: 0.75:
Text: 主旨
第-條 為強為強化國化國泰金融控股股份有限公司(以下簡稱本公司)之資訊安全管 理，建立安全及可信賴之資訊作業環境，確保資保資訊財訊財產之機密性、完 整性及可用性，並提升同仁對資訊安全之認知， 以保以保障員障員工、客戶 、 合作夥伴與本公司之權益，依據｢國泰金融控股股份有限公司資訊安 全政策｣訂｣訂定資訊安全管理辦法(以下簡稱本辦法)。


🔹 Score: 0.74:
Text: 國泰金融控股股份有限公司資訊安全管全管理辦法
110年3月3月22日2日訂定 權責單位：資訊安全部


🔹 Score: 0.73:
Text: 適用對象及範及範圍
第二條 本辦法適法適用對象為本公司所有員工(含(含各級主管、實、實習生及工讀生，以 下同)及)及其他得接觸本公司資訊及訊及資訊財訊財產之合之合作夥伴、委託廠商(含(含 顧問) (以下合稱「受「受委託者」)等。
本辦法適用範用範圍如下，以下合稱資訊財訊財產(Information Assets)：
-、 本公司所有或有或持有之資之資料與文件。
二、 本公司使用之主機、伺服器、個人電腦 (含(含可攜式電腦，以 下同)、行動裝置、終端設備、通訊線路 、儲存設備(含(含可 讀寫CD裝置、外接硬碟 、 USB儲B儲存媒體、記憶卡等，以 下同)，以及與前述相關或是與本公司資司資訊系統相連之硬 體資訊財訊財產。
三、 本公司使用於存於存放資訊財訊財產之實體環境。
四、 本公司使用之資之資訊系統，以及與前述資述資訊系統相關或相 連之軟體資訊財訊財產。
五、 其他本公司所有或有或有權使權使用之資之資訊財訊財產。


🔹 Score: 0.73:
Text: 第十條十條（控管機制）
各單位主管應督導所屬確實遵守本須知，並掌握員工出、缺勤異常動態。

